This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/force.html).  I recommend having both windows open, side-by-side!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    LogVectorOutput,
    MeshcatVisualizer,
    Parser,
    Simulator,
    Sphere,
    StartMeshcat,
)

from manipulation import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Getting some bounce

It's possible to [achieve bouncing](https://www.youtube.com/watch?v=m5UnMWihWC4&t=108s) using the default contact parameters [(models)](https://github.com/mattbev/robot-juggler/tree/main/utils/models). But adjusting those parameters can get you more bounce.  Here is a simulation of a ball being dropped from 1m. 

The two most relevant parameters are the contact stiffness and damping (dissipation).  Search for "point_contact_stiffness" and/or "hunt_crossley_dissipation" in the [MultibodyPlant documentation]([hunt_crossley_dissipation](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html)). The way to specify them in the SDF is documented [here](https://drake.mit.edu/doxygen_cxx/group__multibody__parsing.html).

Note: This example uses TAMSI as the contact solver.  The SAP solver uses slightly different parameters.

In [ ]:
def ball_drop(stiffness, dissipation):
    sdf = f"""
<?xml version="1.0"?>
<sdf version="1.7">
  <model name="bouncing_ball">
    <link name="ball">
      <pose>0 0 0.1 0 0 0</pose> 
      <inertial>
        <mass>1</mass>
        <inertia>
          <ixx>0.001</ixx>
          <iyy>0.001</iyy>
          <izz>0.001</izz>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyz>0</iyz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry>
          <sphere>
            <radius>0.1</radius>
          </sphere>
        </geometry>
        <material>
          <diffuse>1.0 0.34 0.25 1.0</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <sphere>
            <radius>0.1</radius>
          </sphere>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
    </link>
    <joint name="ball_z" type="prismatic">
      <parent>world</parent>
      <child>ball</child>
      <axis>
        <xyz>0 0 1</xyz>
        <limit>
          <effort>0</effort>
        </limit>
      </axis>
    </joint>
    <link name="ground">
      <pose>0 0 -0.5 0 0 0</pose>
      <visual name="visual">
        <geometry>
          <box>
            <size>2 2 1</size>
          </box>
        </geometry>
      </visual>
      <collision name="collision">
        <geometry>
          <box>
            <size>2 2 1</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
    </link>
    <joint name="ground_weld" type="fixed">
      <parent>world</parent>
      <child>ground</child>
      <pose>0 0 0 0 0 0</pose>
    </joint>
  </model>
</sdf>
"""
    builder = DiagramBuilder()
    time_step = 0.001
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModelsFromString(sdf, ".sdf")
    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant.SetPositions(plant_context, [1])

    visualizer.StartRecording()
    simulator.AdvanceTo(3 if running_as_notebook else 0.1)
    visualizer.PublishRecording()

    log = logger.FindLog(context)
    t = log.sample_times()
    z = log.data()[0]
    plt.plot(t, z)
    first_contact = np.argmax(z < 0)
    bounce_height = np.max(z[first_contact:])
    print(f"bounce height = {bounce_height}")


# These are the default values that MultibodyPlant picked (on Nov 16, 2022) for
# this system.
ball_drop(stiffness=19620, dissipation=10.096375546923044)

In [ ]:
ball_drop(stiffness=1000, dissipation=0)

In [ ]:
ball_drop(stiffness=20000, dissipation=0.1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87d8044f-3eb2-4f25-b52f-f51a5eca68b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>